In [2]:
import os
import csv

# Path to the dataset
dataset_base = 'd2/Anomaly-Multiclass-Dataset'

# Output CSV file
output_csv = 'd2_anomaly_muticlass_dataset.csv'

# Splits to process
splits = ['train', 'test', 'val']

# Initialize list to hold video paths and labels
data = []

# Traverse the dataset
for split in splits:
    split_path = os.path.join(dataset_base, split)
    for category in os.listdir(split_path):
        category_path = os.path.join(split_path, category)
        if os.path.isdir(category_path):
            for video_file in os.listdir(category_path):
                if video_file.endswith('.mp4'):  # Ensure only video files are processed
                    video_path = os.path.join(category_path, video_file)
                    label = category  # The folder name is the label
                    data.append([video_path, label])

# Write to CSV
with open(output_csv, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Path to Video', 'Label'])  # Header row
    writer.writerows(data)

print(f"CSV file saved at: {output_csv}")


CSV file saved at: d2_anomaly_muticlass_dataset.csv


In [3]:
d1_models = ["skywalker290/videomae-vivit-d1","skywalker290/videomae-videomae-d1","skywalker290/videomae-timesformer-d1"]

In [7]:
from transformers import pipeline, AutoProcessor, VivitImageProcessor,VideoMAEImageProcessor
d1_models = ["skywalker290/videomae-vivit-d1","skywalker290/videomae-videomae-d1","skywalker290/videomae-timesformer-d1"]
repo = d1_models[2]

video_cls = pipeline(
    task="video-classification",
    model=repo,
    device=0,
    image_processor=repo
)
result = video_cls("d1/Anomaly-detection-Dataset/test/Anomaly/Abuse008_x264.mp4")
print(result)

config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/485M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

[{'score': 0.9994586110115051, 'label': 'Anomaly'}, {'score': 0.000541362096555531, 'label': 'Normal'}]


In [8]:
import csv
from transformers import pipeline
from tqdm import tqdm  # Import tqdm for the progress bar

# Define models
d1_models = [
    "skywalker290/videomae-vivit-d2",
    "skywalker290/videomae-videomae-d2",
    "skywalker290/videomae-timesformer-d2"
]

# Load pipelines for each model
pipelines = [
    pipeline(task="video-classification", model=model, device=0, image_processor=model)
    for model in d1_models
]

# Input and output CSV file paths
input_csv = "d2_anomaly_muticlass_dataset.csv"  # Input CSV with columns: path_to_video, label
output_csv = "d2_model_dataset.csv"  # Output CSV

# Process videos and write results to output CSV
with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['path_to_video', 'vivit_anomaly', 'videomae_anomaly', 'timesformer_anomaly', 
                  'vivit_normal', 'videomae_normal', 'timesformer_normal', 'label']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Wrap the reader with tqdm to show the progress bar
    for row in tqdm(reader, desc="Processing videos", unit="video"):
        video_path = row['Path to Video']  # Ensure the CSV has the correct column name
        label = row['Label']
        
        # Generate scores for both Anomaly and Normal labels from each model
        anomaly_scores = []
        normal_scores = []
        
        for video_cls in pipelines:
            result = video_cls(video_path)
            
            # Extract scores for both Anomaly and Normal labels
            anomaly_score = next((item['score'] for item in result if item['label'] == 'Anomaly'), 0.0)
            normal_score = next((item['score'] for item in result if item['label'] == 'Normal'), 0.0)
            
            anomaly_scores.append(anomaly_score)
            normal_scores.append(normal_score)
    # Write results to the new CSV
        writer.writerow({
            'path_to_video': video_path,
            'vivit_anomaly': anomaly_scores[0],
            'videomae_anomaly': anomaly_scores[1],
            'timesformer_anomaly': anomaly_scores[2],
            'vivit_normal': normal_scores[0],
            'videomae_normal': normal_scores[1],
            'timesformer_normal': normal_scores[2],
            'label': label
        })

print(f"Processing complete. Results saved to {output_csv}")


model.safetensors:   6%|5         | 21.0M/355M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [13]:
import csv
from transformers import pipeline

# Define models
d1_models = [
    "shazab/videomae-base-finetuned-ucf_crime",
]

# Load pipelines for each model
pipelines = [
    pipeline(task="video-classification", model=model, device=0, image_processor=model)
    for model in d1_models
]

# List of anomaly classes
anomaly_classes = [
    "Abuse", "Arson", "Burglary", "Shooting", "Stealing", 
    "Vandalism", "Assault", "Explosion", "Arrest", 
    "Fighting", "Robbery", "Shoplifting", "RoadAccidents"
]

# Input and output CSV file paths
input_csv = "d2_anomaly_muticlass_dataset.csv"  # Input CSV with columns: path_to_video, label
output_csv = "d2_3_model_dataset.csv"  # Output CSV

# Process videos and write results to output CSV
with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    
    # Dynamically create fieldnames based on models and anomaly classes
    fieldnames = ['path_to_video'] + [
        f"{model_name.split('-')[-2]}_{cls}" for model_name in d1_models for cls in anomaly_classes
    ] + ['label']
    
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in reader:
        video_path = row['Path to Video']  # Ensure the CSV has the correct column name
        label = row['Label']
        
        # Initialize result dictionary for this row
        row_results = {'path_to_video': video_path, 'label': label}
        
        # Process each video with each pipeline
        for model_name, video_cls in zip(d1_models, pipelines):
            try:
                print(f"Processing video: {video_path} with model: {model_name}")
                result = video_cls(video_path,top_k=13)
                
                # Extract scores for each anomaly class
                for cls in anomaly_classes:
                    score = next((item['score'] for item in result if item['label'] == cls), 0.0)
                    row_results[f"{model_name.split('-')[-2]}_{cls}"] = score
            except Exception as e:
                print(f"Error processing {video_path} with model {model_name}: {e}")
                # Default all class scores to 0 in case of error
                for cls in anomaly_classes:
                    row_results[f"{model_name.split('-')[-2]}_{cls}"] = 0.0
        
        # Write results for this video to the output CSV
        writer.writerow(row_results)

print(f"Processing complete. Results saved to {output_csv}")


config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/345M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery127_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery136_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery138_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery030_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery094_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery035_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery025_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/R

model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery054_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery070_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery115_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery092_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery049_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery101_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/Robbery/Robbery036_x264.mp4 with model: shazab/videomae-base-finetuned-ucf_crime
Processing video: d2/Anomaly-Multiclass-Dataset/train/R